In [3]:
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf
import pickle
import sys
import yaml

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, TimeDistributed, Bidirectional
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.utils import plot_model
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import load_model
from sklearn import preprocessing
from copy import deepcopy

import datetime
import math
import hashlib
import time
import os
from datagenerator import DataGenerator

# add
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"  # specify which GPU(s) to be used
# add
config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
# # sess = tf.Session(config=config) 
sess = tf.compat.v1.Session(config=config) 

def loadConfig():
#     with open('/home/wesleyjtann/ddos_research/test/light_config_b.yaml', "r") as ymlfile:
    with open('./light_config_hulk.yaml', "r") as ymlfile: #GE, hulk, slowhttp, slowloris, heartbleed
        cfg = yaml.load(ymlfile)
    return cfg

config = loadConfig()
# artefact = 'artefact' #'artefact_b' #

/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/ipykernel_launcher.py:54: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [5]:
df_test = pd.read_csv(config['metadata']['uniqueID'] + '/artefact_hulk/TEST.csv')
df_test

,Source IP,Dest IP,Protocols,Info,Input,Histo
0,0,0,eth:ethertype:arp,Who has 192.168.10.1? Tell 192.168.10.3,169<SEP>169<SEP>169<SEP>169<SEP>8<SEP>169<SEP>...,Who has 192.168.10.1? Tell 192.168.10.3<JOIN>e...
1,0,0,eth:ethertype:arp,Who has 192.168.10.3? Tell 192.168.10.12,169<SEP>169<SEP>169<SEP>169<SEP>169<SEP>169<SE...,Who has 192.168.10.3? Tell 192.168.10.12<JOIN>...
2,0,0,eth:ethertype:arp,Who has 192.168.10.3? Tell 192.168.10.16,169<SEP>169<SEP>169<SEP>169<SEP>169<SEP>169<SE...,Who has 192.168.10.3? Tell 192.168.10.16<JOIN>...
3,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0xda6282 CID: 0001000120d5ad0eb8a...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0xda6282 CID: 0001000120d5ad0eb8a...
4,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0xda6282 CID: 0001000120d5ad0eb8a...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0xda6282 CID: 0001000120d5ad0eb8a...
...,...,...,...,...,...,...
53776,94.31.29.54,192.168.10.14,eth:ethertype:ip:tcp:tls,Application Data,61<SEP>190<SEP>208<SEP>208<SEP>190<SEP>190<SEP...,Application Data<JOIN>eth:ethertype:ip:tcp:tls
53777,94.31.29.54,192.168.10.14,eth:ethertype:ip:tcp,"443 → 52396 [FIN, ACK] Seq=5609 Ack=560 Win=31...",143<SEP>208<SEP>208<SEP>42,"443 → 52396 [FIN, ACK] Seq=5609 Ack=560 Win=31..."
53778,94.31.29.64,192.168.10.8,eth:ethertype:ip:tcp,443 → 52313 [ACK] Seq=6198 Ack=851 Win=32768 L...,185<SEP>130<SEP>130<SEP>130<SEP>54<SEP>238<SEP...,443 → 52313 [ACK] Seq=6198 Ack=851 Win=32768 L...
53779,98.139.199.204,192.168.10.8,eth:ethertype:ip:tcp,443 → 52389 [ACK] Seq=6338 Ack=1167 Win=17920 ...,214<SEP>233,443 → 52389 [ACK] Seq=6338 Ack=1167 Win=17920 ...


In [7]:
df_norm = pd.read_csv(config['metadata']['uniqueID'] + '/artefact_hulk/N2.csv')

print(len(df_norm))
df_norm.head()

25996


,Source IP,Dest IP,Protocols,Info,Input,Histo
0,0,0,eth:ethertype:arp,Who has 192.168.10.1? Tell 192.168.10.19,169<SEP>169<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Who has 192.168.10.1? Tell 192.168.10.19<JOIN>...
1,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0x4dadef CID: 0001000120d5ad0eb8a...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0x4dadef CID: 0001000120d5ad0eb8a...
2,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0x4dadef CID: 0001000120d5ad0eb8a...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0x4dadef CID: 0001000120d5ad0eb8a...
3,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0x4dadef CID: 0001000120d5ad0eb8a...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0x4dadef CID: 0001000120d5ad0eb8a...
4,0,0,eth:ethertype:ipv6:udp:dhcpv6,Solicit XID: 0x4dadef CID: 0001000120d5ad0eb8a...,122<SEP>122<SEP>122<SEP>122<SEP>122<SEP>122<SE...,Solicit XID: 0x4dadef CID: 0001000120d5ad0eb8a...


In [8]:
df_norm = df_norm.sample(frac=1)

# df_normN1 = df_norm[:35000] #[:35263]
df_normN2 = df_norm[-2996:] #[35264:]

## Save new N1
# df_normN1.to_csv('../1_testmimic_caida07/ids_test2/artefact/N1_new.csv',index=False)

In [9]:
len(df_normN2)

2996

In [10]:
## True normals

# df_norm = df_test[(df_test['Attack']==0)]
# df_norm = df_test

df_normN2['Attack']=0

print("length df_norm: ",  len(df_normN2)) 

length df_norm:  2996


/home/wesleyjtann/miniconda3/envs/ddos/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df_normN2.to_csv('../1_testmimic_cicWed/ids_test2/artefact_hulk/TESTnorm.csv', index=False)

In [ ]:
3262/.2